<a href="https://colab.research.google.com/github/vboastom/Coursera_Capstone/blob/main/Capstone_Project_The_Battle_of_Neighborhoods_(Week_1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IBM Data Science - Professional Certificate
## Capstone Projects - Battle of Neighborhoods
### Author: Vitor Villas Boas

In [2]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans
import folium # map rendering library
print('Libraries imported.')

Libraries imported.


In [13]:
CLIENT_ID = 'BRQRODSZVFCG1SGCRV5IBOH3VGD0TDDJUCNTP3HW2Q23XEKE' # your Foursquare ID
CLIENT_SECRET = 'OPQVXCAULA3B0SCVMMK1KNEKCXSFMGIE40C4UKTB1IFVGK5Q' # your Foursquare Secret
VERSION = '20180604'
ACCESS_TOKEN = 'AIMBO2C1MFQ1P03XG3ZI3JRL3Z22MOEHZYOCEMZMYI325RJ4' # your access token to user.id
LIMIT = 30

In [3]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
print('Data downloaded!')

Data downloaded!


In [7]:
with open('newyork_data.json') as json_data:
    nyc_data = json.load(json_data)

In [11]:
neighborhoods_data = nyc_data['features']
neighborhoods_data[0]

{'geometry': {'coordinates': [-73.84720052054902, 40.89470517661],
  'type': 'Point'},
 'geometry_name': 'geom',
 'id': 'nyu_2451_34572.1',
 'properties': {'annoangle': 0.0,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661],
  'borough': 'Bronx',
  'name': 'Wakefield',
  'stacked': 1},
 'type': 'Feature'}

In [12]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)